In [ ]:
from datetime import datetime, timedelta
import glob
import os

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame()

path_to_json = '../../runs/' 

json_pattern = os.path.join(path_to_json, '*.json')
file_list = glob.glob(json_pattern)

for file in file_list:
    data = pd.read_json(file)
    df = pd.concat([df, data], ignore_index = True)

In [ ]:
df['departure_date_dt'] = pd.to_datetime(df['departure_date'].apply(lambda x: f"{x} 2024"))
df['return_date_dt'] = pd.to_datetime(df['return_date'].apply(lambda x: f"{x} 2024"))

df['departure_dt'] = pd.to_datetime(df.apply(lambda row: f"{row['departure']} {row['departure_date']} 2024", axis=1))
df['flight_duration_dt'] =  pd.to_timedelta(df['duration'])
df['landing_origin_dt'] = df['departure_dt'] + df['flight_duration_dt']

In [ ]:
df['trip_duration'] = df['return_date_dt'] - df['departure_date_dt']
df['trip_duration_days'] = df['trip_duration'].dt.days

In [ ]:
df['flight_duration_dt'] =  pd.to_timedelta(df['duration'])

In [ ]:
unique_trip_duration_days = df['trip_duration_days'].unique()
trip_duration_median = np.median(unique_trip_duration_days)
df['departure_date_with_jitter'] = df.apply(
    lambda row: row['departure_date_dt'] + timedelta(hours = 2*(row['trip_duration_days'] - trip_duration_median)), 
    axis=1
)

In [ ]:
px.scatter(df, x='departure_date_with_jitter', y='price_unit', color='nonstop?', opacity=0.5)

In [ ]:
df

In [ ]:
df_sub = df[df['departure_date_dt'] == datetime(2024, 8, 30)]


fig = go.Figure()
for _, row in df_sub.iterrows():
    fig.add_trace(
        go.Scatter(
            x=[row['departure_dt'], row['landing_origin_dt']],
            y=[row['price_unit'], row['price_unit']],
        )
    )

fig.update_layout(
    autosize=True,
    height=750,
)

fig.show()

# TODOs

- add datetime selector for second plot
- add filter for direct flights
- add filter for price
- add selector for min and max connection time